## Deliverable 2. Create a Customer Travel Destinations Map.

In [20]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

import os
os.system('say "your program has finished"')

0

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()
# os.system('say "your program has finished"')

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pevek,RU,69.7008,170.3133,32.43,86,99,28.32,overcast clouds
1,1,Broome,US,42.2506,-75.8330,65.61,86,0,1.01,clear sky
2,2,Cap Malheureux,MU,-19.9842,57.6142,72.46,71,37,8.99,scattered clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,77.18,32,2,7.00,clear sky
4,4,Sept-Iles,CA,50.2001,-66.3821,57.06,90,96,5.12,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_unclean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_unclean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Port Alfred,ZA,-33.5906,26.8910,77.18,32,2,7.00,clear sky
8,8,Baracoa,CU,20.3467,-74.4958,78.66,89,3,1.63,clear sky
12,12,Atuona,PF,-9.8000,-139.0333,77.65,75,1,18.16,clear sky
26,26,Kapaa,US,22.0752,-159.3190,80.58,84,100,16.11,overcast clouds
29,29,Emba,KZ,48.8264,58.1442,87.58,29,11,4.99,few clouds
31,31,Butaritari,KI,3.0707,172.7902,81.03,71,100,2.71,light rain
35,35,Gat,IL,31.6100,34.7642,88.39,30,0,8.46,clear sky
39,39,Marrakesh,MA,31.6315,-8.0083,77.07,65,40,4.61,scattered clouds
44,44,Kavieng,PG,-2.5744,150.7967,82.33,78,65,14.38,broken clouds
53,53,Naze,JP,28.3667,129.4833,81.86,81,100,11.72,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_unclean_df.count()

City_ID                242
City                   242
Country                240
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [56]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_unclean_df.dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Port Alfred,ZA,-33.5906,26.8910,77.18,32,2,7.00,clear sky
8,8,Baracoa,CU,20.3467,-74.4958,78.66,89,3,1.63,clear sky
12,12,Atuona,PF,-9.8000,-139.0333,77.65,75,1,18.16,clear sky
26,26,Kapaa,US,22.0752,-159.3190,80.58,84,100,16.11,overcast clouds
29,29,Emba,KZ,48.8264,58.1442,87.58,29,11,4.99,few clouds


In [57]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Port Alfred,ZA,77.18,clear sky,-33.5906,26.8910,
8,Baracoa,CU,78.66,clear sky,20.3467,-74.4958,
12,Atuona,PF,77.65,clear sky,-9.8000,-139.0333,
26,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,
29,Emba,KZ,87.58,few clouds,48.8264,58.1442,
31,Butaritari,KI,81.03,light rain,3.0707,172.7902,
35,Gat,IL,88.39,clear sky,31.6100,34.7642,
39,Marrakesh,MA,77.07,scattered clouds,31.6315,-8.0083,
44,Kavieng,PG,82.33,broken clouds,-2.5744,150.7967,
53,Naze,JP,81.86,overcast clouds,28.3667,129.4833,


In [58]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name,
    # if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

os.system('say "your program has finished"')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


0

In [66]:
# 7. Drop the rows where there is no Hotel Name.
# get names of indexes for which column 'Hotel Name' has value ''
index_names = hotel_df[hotel_df['Hotel Name'] == '' ].index

# drop these row indexes from dataFrame
hotel_df.drop(index_names, inplace = True)

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Port Alfred,ZA,77.18,clear sky,-33.5906,26.8910,The Halyards Hotel
8,Baracoa,CU,78.66,clear sky,20.3467,-74.4958,Hostal Rio Miel
12,Atuona,PF,77.65,clear sky,-9.8000,-139.0333,Villa Enata
26,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
31,Butaritari,KI,81.03,light rain,3.0707,172.7902,Isles Sunset Lodge


In [67]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [86]:
# 9. Using the template add city name, the country code,
# the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the
# formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [87]:
# 11a. Add a marker layer for each city to the map.
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))